In [ ]:
%load_ext autoreload
%autoreload 2

import sklearn
import os, sys
import numpy as np
import tensorflow as tf
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from scipy import interpolate
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

sys.path.insert(1, os.path.join(sys.path[0], ".."))
sys.path.append('../../')
from reproduction.pipeline.load import load_data
from reproduction import analysis

In [ ]:
def gridsearch(start, step, stop, max_samples=5000, sample_steps=4):
    with open(ENCODER_DEF,"r") as f:
            encoder = tf.keras.models.model_from_json(f.read())
    encoder.load_weights(ENCODER_WEIGHTS)
    
    samples = int(np.logspace(np.log10(2), np.log10(max_samples), num=sample_steps))
    for i in samples:
        data = analysis.AEData(load_data(DATA, encoder.input_shape[1:]), n=i)
        data.add_encoder(encoder)
        search_results = [] # Force initialization
        for j in range(start, stop, step):
            N_CLUSTERS = j
            ag1 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs[:int(i/2)])
            ag2 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs)
            minfo = sklearn.metrics.adjusted_mutual_info_score(ag1, ag2[:int(i/2)])
            # print('Clusters: ',N_CLUSTERS,' Mutual information: ', minfo)
            search_results.append((i, N_CLUSTERS, minfo))
    return search_results
    

In [ ]:
DATA = "/project/foster/clouds/data/2015_05/*.tfrecord"
ENCODER_DEF = "/home/rlourenco/rdcep_clouds/output/m9-22_oceans/encoder.json"
ENCODER_WEIGHTS = "/home/rlourenco/rdcep_clouds/output/m9-22_oceans/encoder.h5"
# N_CLUSTERS = 10

In [ ]:
result = gridsearch(2,1,500)
N_CLUSTERS = result[0]
HiMI = result[1]
print('Best result: Clusters# ',N_CLUSTERS,' Mutual information# ', HiMI)

In [ ]:
with open(ENCODER_DEF,"r") as f:
    encoder = tf.keras.models.model_from_json(f.read())
encoder.load_weights(ENCODER_WEIGHTS)

In [ ]:
data = analysis.AEData(load_data(DATA, encoder.input_shape[1:]), n=5000)
data.add_encoder(encoder)

In [ ]:
ag1 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs[:2500])
ag2 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs)

In [ ]:
# Number of bins
nbins=N_CLUSTERS

plt.figure(figsize=(8,8))
plt.hist2d(ag1, ag2[:2500], bins=nbins);
plt.title("Cluster overlap")
plt.xticks(range(0, nbins, 5)); plt.yticks(range(0, nbins, 5));
plt.xlabel("ag 5000"); plt.ylabel("ag 2500");
plt.colorbar()
print('Mutual Information score: ',sklearn.metrics.adjusted_mutual_info_score(ag1, ag2[:2500]))

In [ ]:
overlap = data.imgs[:2500,:,:,0][ag1 == ag2[:2500]]
overlap_c = ag1[ag1 == ag2[:2500]]



In [ ]:
fig, ax = plt.subplots(5, 8, figsize=(16, 10))

clu = data.imgs[:2500,:,:,0][ag1 == 3]

for i, a in enumerate(ax.ravel()):
    a.imshow(clu[i], cmap="bone")
    a.set_xticks([])
    a.set_yticks([])



In [ ]:

from scipy.cluster.hierarchy import dendrogram

In [ ]:
def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)



Well that's not useful...

In [ ]:
plt.figure(figsize=(500,100))
plot_dendrogram(ag2, labels=ag2.labels_)

In [ ]:
ag2.